In [1]:
import re

class myTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        return text

In [2]:
with open("the-verdict.txt", 'r') as f:
    raw_text  =f.read()

preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [3]:
all_words = sorted(set(preprocessed))
vocab = {token:integer for integer, token in enumerate(all_words)}

tokenizer = myTokenizer(vocab)

text = '"It\'s the last he painted, you know\", Mrs. Gisburn said with pardonable pride.'
ids = tokenizer.encode(text)
print(ids)

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 1, 5, 69, 7, 39, 873, 1136, 773, 812, 7]


In [4]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know", Mrs. Gisburn said with pardonable pride.


In [5]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(type(vocab.items()))

<class 'dict_items'>


In [6]:
print(list(vocab.items())[-5:])

[('younger', 1156), ('your', 1157), ('yourself', 1158), ('<|endoftext|>', 1159), ('<|unk|>', 1160)]


In [7]:
class myTokenizer2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.int_to_str else "<|unk|>" for item in preprocessed] 
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        return text

In [8]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
text = "hawk tuah"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[40624, 12777, 993]


In [9]:
strings = tokenizer.decode(integers)
print(strings)

hawk tuah


In [12]:
encoded_text = tokenizer.encode(raw_text)
encoded_sample = encoded_text[50:]  # remove the first 50 encodings

context_size = 4

x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

for i in range(1,context_size+1):
    context = encoded_sample[:i]
    desired = encoded_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, max_length , step):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - max_length, step):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [14]:
def createDataLoader(text, batch_size = 4, max_length = 256, step=128, shuffle = True, drop_last = True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(text, tokenizer, max_length, step)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

dataloader = createDataLoader(raw_text, batch_size=1, max_length=4, step=1, shuffle=False)
data_iteration = iter(dataloader)
first_batch = next(data_iteration)
print(first_batch) 

second_batch = next(data_iteration)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [15]:
dataloader = createDataLoader(raw_text, batch_size=8, max_length=4, step=4, shuffle=False)
data_iteration = iter(dataloader)
inputs, targets = next(data_iteration)
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')

Inputs: tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Create token embeddings

In [18]:
vocab_size = 50257
output_dim = 256

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloaders = createDataLoader(raw_text, batch_size=8, max_length=max_length, step=4, shuffle=False)
data_iter = iter(dataloaders)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)
token_embeddings = embedding_layer(inputs)
print("Token embeddings dim", token_embeddings.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])
Token embeddings dim torch.Size([8, 4, 256])


#### Absolute embedding

OpenAI's GPT uses absolute positional embedding to tell the model the positions of the tokens. Let's replicate that:

In [19]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
position_embeddings = pos_embedding_layer(torch.arange(context_length))
print(position_embeddings.shape)

torch.Size([4, 256])


In [20]:
input_embeddings = token_embeddings + position_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
